## Librerias

In [16]:
import matplotlib
matplotlib.use('Agg') # Fuerza un backend que no requiere ventana gráfica

# ... el resto de tus imports
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import accuracy_score

# --- IMPORTAR LOS GLADIADORES ---
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import GaussianNB


## carga de archivos

In [17]:
df = pd.read_csv('datos/DATASET_LIKERT_NUMERICO.csv') # Tu archivo final
df.head(5)

,Carrera_Target,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8,Q9,...,Q16,Q17,Q18,Q19,Q20,Q21,Q22,Q23,Q24,Q25
0,Software,1,3,1,2,1,4,1,1,5,...,4,4,2,3,2,3,5,2,5,f
1,Software,3,5,5,4,5,4,3,1,5,...,3,4,1,2,4,3,3,1,4,a
2,Software,2,5,1,2,3,3,1,4,3,...,2,5,3,3,3,1,3,1,4,b
3,Software,2,4,3,3,2,1,3,3,5,...,2,1,1,2,2,1,1,2,4,a
4,Software,3,5,1,3,3,3,4,3,5,...,2,5,3,3,2,1,3,1,5,a


## Preprocesamiento (Codificar letras a números)
### IMPORTANTE: SVM y KNN son sensibles a la escala, así que a veces ayuda escalar, pero como todas tus preguntas son del 0 al 4 (incisos), ya están en la misma escala.

In [18]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# 1. CARGA (Asegúrate de que la ruta sea correcta)
try:
    df = pd.read_csv('datos/DATASET_LIKERT_NUMERICO.csv')
except:
    df = pd.read_csv('DATASET_NUEVO_LIKERT.csv')

print("1. Datos originales cargados. Ejemplo Q25:", df['Q25'].unique())

# 2. LIMPIEZA FORZADA (Específica para Q25 y Target)
# No confiamos en bucles automáticos, vamos directo a la yugular
le_q25 = LabelEncoder()
df['Q25'] = le_q25.fit_transform(df['Q25'].astype(str))

le_target = LabelEncoder()
df['Carrera_Target'] = le_target.fit_transform(df['Carrera_Target'])

print("2. Traducción completada. Ejemplo Q25 (ahora números):", df['Q25'].unique())

# 3. CREACIÓN DE X e Y
# Eliminamos cualquier columna que no sea una Pregunta (Q1...Q25)
X = df.drop(['Carrera_Target'], axis=1)
y = df['Carrera_Target']

# Verificación de seguridad
if X['Q25'].dtype == 'object':
    print("⚠️ ALERTA: Q25 sigue siendo texto. Algo raro pasa.")
else:
    print("✅ TODO EN ORDEN: Q25 ya es numérico.")

# 4. SPLIT
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"3. Split listo. X_train shape: {X_train.shape}")
print("-" * 30)
print("Vista previa final de X (deberían ser puros números):")
print(X.head())

1. Datos originales cargados. Ejemplo Q25: <StringArray>
['f', 'a', 'b', 'c', 'e', 'd']
Length: 6, dtype: str
2. Traducción completada. Ejemplo Q25 (ahora números): [5 0 1 2 4 3]
✅ TODO EN ORDEN: Q25 ya es numérico.
3. Split listo. X_train shape: (1680, 25)
------------------------------
Vista previa final de X (deberían ser puros números):
   Q1  Q2  Q3  Q4  Q5  Q6  Q7  Q8  Q9  Q10  ...  Q16  Q17  Q18  Q19  Q20  Q21  \
0   1   3   1   2   1   4   1   1   5    1  ...    4    4    2    3    2    3   
1   3   5   5   4   5   4   3   1   5    4  ...    3    4    1    2    4    3   
2   2   5   1   2   3   3   1   4   3    3  ...    2    5    3    3    3    1   
3   2   4   3   3   2   1   3   3   5    3  ...    2    1    1    2    2    1   
4   3   5   1   3   3   3   4   3   5    4  ...    2    5    3    3    2    1   

   Q22  Q23  Q24  Q25  
0    5    2    5    5  
1    3    1    4    0  
2    3    1    4    1  
3    1    2    4    0  
4    3    1    5    0  

[5 rows x 25 columns]


## Definir la lista de modelos

In [19]:
modelos = [
    ('Random Forest', RandomForestClassifier(n_estimators=100, random_state=42)),
    ('SVM (Kernel RBF)', SVC(gamma='auto')), # Bueno para fronteras curvas
    ('KNN (5 Vecinos)', KNeighborsClassifier(n_neighbors=5)),
    ('Gradient Boosting', GradientBoostingClassifier(random_state=35)),
    ('Naive Bayes', GaussianNB()) # El más rápido y probabilístico
]

resultados = []
nombres = []

print(f"{'Modelo':<20} | {'Precisión':<10}")
print("-" * 35)

Modelo               | Precisión 
-----------------------------------


## Entrenar y Evaluar uno por uno

In [20]:
for nombre, modelo in modelos:
    # Entrenar
    modelo.fit(X_train, y_train)
    
    # Predecir
    y_pred = modelo.predict(X_test)
    
    # Calcular precisión
    acc = accuracy_score(y_test, y_pred)
    
    resultados.append(acc)
    nombres.append(nombre)
    
    print(f"{nombre:<20} | {acc:.2%}")

Random Forest        | 91.81%
SVM (Kernel RBF)     | 91.25%
KNN (5 Vecinos)      | 86.25%
Gradient Boosting    | 91.11%
Naive Bayes          | 91.67%


## Graficar los resultados

In [21]:
import matplotlib.pyplot as plt
import seaborn as sns

# Configuración de la gráfica
plt.figure(figsize=(10, 6))
colores = sns.color_palette("viridis", len(modelos))

# Usamos hue=nombres y legend=False para cumplir con la nueva versión de Seaborn
sns.barplot(x=nombres, y=resultados, hue=nombres, palette=colores, legend=False)

plt.ylim(0.7, 1.0) 
plt.title('Comparación de Modelos: Precisión por Algoritmo', fontsize=14)
plt.ylabel('Precisión (Accuracy)')
plt.xticks(rotation=15)
plt.grid(axis='y', linestyle='--', alpha=0.7)

# Agregar las etiquetas de porcentaje sobre las barras
for i, v in enumerate(resultados):
    plt.text(i, v + 0.01, f"{v:.1%}", ha='center', fontweight='bold')

plt.tight_layout()

# LA LÍNEA MÁS IMPORTANTE:
plt.savefig('resultados_torneo_modelos.png', dpi=300) # dpi=300 para que se vea súper nítida
print("¡Éxito! La gráfica se guardó como 'resultados_torneo_modelos.png' en tu carpeta actual.")

# Si estás en VS Code, esto ayudará a que se vea en el notebook también
plt.show()

¡Éxito! La gráfica se guardó como 'resultados_torneo_modelos.png' en tu carpeta actual.


/tmp/ipykernel_52054/471040450.py:28: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


In [22]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

# Asegúrate de que 'modelos' tiene el Random Forest entrenado en la posición 0
# Si no, búscalo en tu lista. Por lo general es el primero: modelos[0][1]
rf_model = modelos[0][1] 

# Obtener la importancia de cada pregunta
importancias = rf_model.feature_importances_
nombres_preguntas = X.columns

# Crear un DataFrame para ordenarlos
df_imp = pd.DataFrame({'Pregunta': nombres_preguntas, 'Importancia': importancias})
df_imp = df_imp.sort_values('Importancia', ascending=False).head(10) # Top 10

# Graficar
plt.figure(figsize=(10, 6))
sns.barplot(data=df_imp, x='Importancia', y='Pregunta', palette='viridis')

plt.title('Top 10: ¿Qué preguntas definen tu carrera?', fontsize=14)
plt.xlabel('Nivel de Influencia en la Decisión')
plt.ylabel('Pregunta del Test')
plt.grid(axis='x', linestyle='--', alpha=0.5)
plt.show() # O plt.savefig('importancia_preguntas.png')

/tmp/ipykernel_52054/808003548.py:19: FutureWarning: 

Passing `palette` without assigning `hue` is deprecated and will be removed in v0.14.0. Assign the `y` variable to `hue` and set `legend=False` for the same effect.

  sns.barplot(data=df_imp, x='Importancia', y='Pregunta', palette='viridis')
/tmp/ipykernel_52054/808003548.py:25: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show() # O plt.savefig('importancia_preguntas.png')


## Pruebas

### Respuestas softere

In [24]:
# --- BLOQUE DE PRUEBA INTERACTIVA (Pégalo al final de tu notebook local) ---
import numpy as np

# 1. TUS RESPUESTAS (Escala 1 al 5)
# Modifica estos números con lo que TÚ responderías realmete.
mis_respuestas = {
    # Aptitudes (1=Se me complica, 5=Es lo mío)
    'Q1': 4, 'Q2': 5, 'Q3': 2, 'Q4': 3, 
    'Q5': 2, 'Q6': 3, 'Q7': 5, 'Q8': 1,
    
    # Intereses (1=Me aburre, 5=Me apasiona)
    'Q9': 5, 'Q10': 4, 'Q11': 1, 'Q12': 2,
    'Q13': 1, 'Q14': 4, 'Q15': 3, 'Q16': 2,
    
    # Futuro (1=Jamás, 5=Diario)
    'Q17': 5, 'Q18': 2, 'Q19': 1, 'Q20': 1,
    'Q21': 2, 'Q22': 3, 'Q23': 1, 'Q24': 5,
    
    # Materia Favorita (Pon la letra: 'a', 'b', 'c', 'd', 'e', 'f')
    'Q25': 'a' 
}

# ---------------------------------------------------------
# NO TOQUES NADA DE AQUÍ PARA ABAJO (Lógica del modelo)
# ---------------------------------------------------------

# Preparar vector
vector_entrada = []
for i in range(1, 25):
    vector_entrada.append(mis_respuestas[f'Q{i}'])

# Codificar la letra de la Q25 usando el encoder que ya entrenaste arriba
# NOTA: Asegúrate de que 'le_q25' es el nombre de tu variable encoder. 
# Si arriba le pusiste otro nombre (ej. label_encoder_materias), cámbialo aquí.
try:
    val_q25 = le_q25.transform([mis_respuestas['Q25']])[0]
    vector_entrada.append(val_q25)
except:
    print("Error: Revisa que la letra de Q25 sea válida (a-f) y que 'le_q25' exista.")

# Convertir a matriz
vector_final = np.array([vector_entrada])

# Usar el mejor modelo de tu lista (Por ejemplo, el Gradient Boosting o Naive Bayes)
# Buscamos el modelo en tu lista 'modelos'. 
# Ajusta el índice [3] o [4] según cuál haya ganado el torneo.
mejor_modelo = modelos[4][1] # [4] suele ser Naive Bayes, [3] Gradient Boosting

# Predecir
try:
    prediccion_num = mejor_modelo.predict(vector_final)[0]
    # Decodificar el número a nombre de carrera
    nombre_carrera = le_target.inverse_transform([prediccion_num])[0]

    print("\n" + "="*40)
    print(f"🔮 PREDICCIÓN DEL MODELO: {nombre_carrera.upper()}")
    print("="*40)
except Exception as e:
    print(f"Ocurrió un error al predecir: {e}")
    print("Asegúrate de haber corrido todas las celdas de entrenamiento anteriores.")


🔮 PREDICCIÓN DEL MODELO: SOFTWARE


/home/armando/tesis/tesis/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(


In [25]:
# --- BLOQUE DE PRUEBA INTERACTIVA (Pégalo al final de tu notebook local) ---
import numpy as np

# 1. TUS RESPUESTAS (Escala 1 al 5)
# Modifica estos números con lo que TÚ responderías realmete.
# PERFIL SIMULADO: INGENIERO CIVIL / ARQUITECTO
mis_respuestas = {
    # Odia la compu, ama construir
    'Q1': 5, 'Q2': 1, 'Q3': 5, 'Q4': 4,  # Mates, No PC, Planos, Herramientas
    'Q5': 3, 'Q6': 4, 'Q7': 3, 'Q8': 5,  # 3D Espacial (Clave para civil)
    
    # Intereses
    'Q9': 1, 'Q10': 1, 'Q11': 5, 'Q12': 3, # No Code, SI Construcción
    'Q13': 2, 'Q14': 2, 'Q15': 3, 'Q16': 1,
    
    # Futuro
    'Q17': 1, 'Q18': 3, 'Q19': 5, 'Q20': 2, # JAMAS programar, SI Obra
    'Q21': 3, 'Q22': 2, 'Q23': 5, 'Q24': 1, # Estructuras SI
    
    # Materia: Construcción
    'Q25': 'c' 
}

# ---------------------------------------------------------
# NO TOQUES NADA DE AQUÍ PARA ABAJO (Lógica del modelo)
# ---------------------------------------------------------

# Preparar vector
vector_entrada = []
for i in range(1, 25):
    vector_entrada.append(mis_respuestas[f'Q{i}'])

# Codificar la letra de la Q25 usando el encoder que ya entrenaste arriba
# NOTA: Asegúrate de que 'le_q25' es el nombre de tu variable encoder. 
# Si arriba le pusiste otro nombre (ej. label_encoder_materias), cámbialo aquí.
try:
    val_q25 = le_q25.transform([mis_respuestas['Q25']])[0]
    vector_entrada.append(val_q25)
except:
    print("Error: Revisa que la letra de Q25 sea válida (a-f) y que 'le_q25' exista.")

# Convertir a matriz
vector_final = np.array([vector_entrada])

# Usar el mejor modelo de tu lista (Por ejemplo, el Gradient Boosting o Naive Bayes)
# Buscamos el modelo en tu lista 'modelos'. 
# Ajusta el índice [3] o [4] según cuál haya ganado el torneo.
mejor_modelo = modelos[4][1] # [4] suele ser Naive Bayes, [3] Gradient Boosting

# Predecir
try:
    prediccion_num = mejor_modelo.predict(vector_final)[0]
    # Decodificar el número a nombre de carrera
    nombre_carrera = le_target.inverse_transform([prediccion_num])[0]

    print("\n" + "="*40)
    print(f"🔮 PREDICCIÓN DEL MODELO: {nombre_carrera.upper()}")
    print("="*40)
except Exception as e:
    print(f"Ocurrió un error al predecir: {e}")
    print("Asegúrate de haber corrido todas las celdas de entrenamiento anteriores.")


🔮 PREDICCIÓN DEL MODELO: CIVIL


/home/armando/tesis/tesis/lib/python3.12/site-packages/sklearn/utils/validation.py:2691: UserWarning: X does not have valid feature names, but GaussianNB was fitted with feature names
  warnings.warn(
